In [1]:
@file:DependsOn("com.toldoven.aoc:aoc-kotlin-notebook:1.1.2")

import com.toldoven.aoc.notebook.AocClient

In [2]:
val aoc = AocClient.fromFile().interactiveDay(2024, 10)

In [3]:
val input = aoc.input()

In [4]:
enum class Direction(val x: Int, val y: Int) {
    UP(0, -1), DOWN(0, 1), LEFT(-1, 0), RIGHT(1, 0)
}

data class Vec2(val x: Int, val y: Int) {
    fun move(d: Direction) = Vec2(x + d.x, y + d.y)
    fun around() = Direction.entries.map { move(it) }
}

class Grid(private val data: List<List<Int>>) {
    val rangeX = 0..data[0].lastIndex
    val rangeY = 0..data.lastIndex
    val points = rangeY.flatMap { y -> rangeX.map { x -> Vec2(x, y) } }

    operator fun contains(p: Vec2) = p.x in rangeX && p.y in rangeY
    operator fun get(p: Vec2): Int = if (p in this) data[p.y][p.x] else error("Not a valid grid position")
}

fun trailheads(grid: Grid): List<Vec2> {
    return buildList<Vec2> {
        addAll(grid.points.filter { grid[it] == 0 })
    }
}

In [5]:
val example = """89010123
78121874
87430965
96549874
45678903
32019012
01329801
10456732"""

val exampleGrid = Grid(example.lines().map { it.toCharArray().map { it.digitToInt() } })

In [6]:
fun pathSections(grid: Grid): Map<Vec2, List<Vec2>> = buildMap {
    grid.points.map { point -> put(point, point.around().filter { it in grid && grid[it] == grid[point] + 1 }) }
}

val examplePathSections = pathSections(exampleGrid)
examplePathSections

{Vec2(x=0, y=0)=[Vec2(x=1, y=0)], Vec2(x=1, y=0)=[], Vec2(x=2, y=0)=[Vec2(x=2, y=1), Vec2(x=3, y=0)], Vec2(x=3, y=0)=[Vec2(x=3, y=1)], Vec2(x=4, y=0)=[Vec2(x=4, y=1), Vec2(x=3, y=0), Vec2(x=5, y=0)], Vec2(x=5, y=0)=[Vec2(x=6, y=0)], Vec2(x=6, y=0)=[Vec2(x=7, y=0)], Vec2(x=7, y=0)=[Vec2(x=7, y=1)], Vec2(x=0, y=1)=[Vec2(x=0, y=0), Vec2(x=0, y=2), Vec2(x=1, y=1)], Vec2(x=1, y=1)=[Vec2(x=1, y=0)], Vec2(x=2, y=1)=[Vec2(x=3, y=1)], Vec2(x=3, y=1)=[Vec2(x=3, y=2)], Vec2(x=4, y=1)=[Vec2(x=3, y=1)], Vec2(x=5, y=1)=[Vec2(x=5, y=2)], Vec2(x=6, y=1)=[Vec2(x=5, y=1)], Vec2(x=7, y=1)=[Vec2(x=7, y=2)], Vec2(x=0, y=2)=[Vec2(x=0, y=3)], Vec2(x=1, y=2)=[Vec2(x=1, y=1), Vec2(x=0, y=2)], Vec2(x=2, y=2)=[Vec2(x=2, y=3)], Vec2(x=3, y=2)=[Vec2(x=3, y=3), Vec2(x=2, y=2)], Vec2(x=4, y=2)=[Vec2(x=4, y=1)], Vec2(x=5, y=2)=[], Vec2(x=6, y=2)=[Vec2(x=6, y=1), Vec2(x=6, y=3)], Vec2(x=7, y=2)=[Vec2(x=6, y=2)], Vec2(x=0, y=3)=[], Vec2(x=1, y=3)=[Vec2(x=1, y=2)], Vec2(x=2, y=3)=[Vec2(x=2, y=4), Vec2(x=1, y=3)], Vec2(x

In [7]:
fun findTrails(pathSections: Map<Vec2, List<Vec2>>, trailhead: Vec2): Set<Vec2> {
    val visited = mutableSetOf<Vec2>()
    val queue = ArrayDeque<Vec2>()
    queue.add(trailhead)
    while (queue.isNotEmpty()) {
        val point = queue.removeFirst()
        if (point !in visited) {
            visited.add(point)
            pathSections[point]?.let { walkable -> queue.addAll(walkable) }
        }
    }
    return visited
}

In [8]:
val exampleTrailheads = trailheads(exampleGrid)
val exampleScore = exampleTrailheads.map { findTrails(examplePathSections, it).filter { exampleGrid[it] == 9 }.count() }.sum()
exampleScore

36

In [9]:
val inputGrid = Grid(input.lines().map { it.toCharArray().map { it.digitToInt() } })
val trailheads = trailheads(inputGrid)
val pathSections = pathSections(inputGrid)

In [10]:
val score = trailheads.map { findTrails(pathSections, it).filter { inputGrid[it] == 9 }.count() }.sum()
score

548

In [11]:
aoc.submitPartOne(548)

The answer is correct! Your answer: 548 You already completed this part.

In [14]:
fun buildTrails(grid: Grid, trailhead: Vec2): Set<List<Vec2>> {
    val trails = mutableSetOf<List<Vec2>>()
    val incomplete = ArrayDeque<List<Vec2>>()
    incomplete.add(listOf(trailhead))
    while (incomplete.isNotEmpty()) {
        val trail = incomplete.removeFirst()
        val point = trail.last()
        val height = grid[point]
        if (height == 9) trails += trail
        else incomplete += point.around().filter { it in grid && grid[it] == height + 1 }.map { trail + it }
    }
    return trails
}

In [15]:
val exampleRating = exampleTrailheads.sumOf { trailhead ->
    buildTrails(exampleGrid, trailhead).count()
}
exampleRating

81

In [16]:
val rating = trailheads.sumOf { trailhead ->
    buildTrails(inputGrid, trailhead).count()
}
rating

1252

In [17]:
aoc.submitPartTwo(rating)

Your answer: 1252. 
 That's the right answer! You are one gold star closer to finding the Chief Historian. 
 You have completed Day 10! You can [Share on Bluesky Twitter Mastodon ] this victory or [Return to Your Advent Calendar] .